# Natural Language Processing

**Referenznotebook zum Thema Natural Language Processing. Für die vollständige Erklärungen bitte auch das Video nutzen.**


In dieser Vorlesung werden wir anhand eines Beispiels veranschaulichen wie man R für Natural Language Processing verwenden kann. Es sind keine Aufgaben und auch kein Projekt beigefügt da dieses Thema zu umfangreich ist. Du wirst vermutlich auch bemerken, dass R nicht unbedingt die beste Wahl für NLP ist. Python eignet sich hier wegen dem Library Support meist besser.
Falls du dennoch mehr Informationen möchtest, kannst du diese [hier](http://www.mjdenny.com/Text_Processing_In_R.html) finden


## Installation der benötigten Libraries 

Wir benötigen die folgenden Libraries:
* tm
* twitteR
* wordcloud
* RColorBrewer
* e1017
* class

Du kannst sie mit diesem Code installieren (erst auskommentieren):

In [54]:
#install.packages('tm',repos='http://cran.us.r-project.org')
#install.packages('twitteR',repos='http://cran.us.r-project.org')
#install.packages('wordcloud',repos='http://cran.us.r-project.org')
#install.packages('RColorBrewer',repos='http://cran.us.r-project.org')
#install.packages('e1017',repos='http://cran.us.r-project.org')
#install.packages('class',repos='http://cran.us.r-project.org')

_____
# Eine Twitter App erstellen


Für dieses Projekt benötigst du einen Twitter Account und eine Twitter Applikation. Hier die benötigten Schritte: 

1. Einen Account auf Twitter erstellen
2. Eine neue App erstellen auf: https://apps.twitter.com/
3. Eventuell musst du es auf eine persönliche URL verweisen. Hierfür kannst du z.B. eine Wordpress Seite oder etwas Ähnliches erstellen. 
4. Erhalte deine Keys bei dem **Keys and Access Tokens** tab
5. Verwende diese Keys mit der TwitteR Library: 

        getTwitterOAuth(consumer_key, consumer_secret)

____
## Übersicht der Regular Expression 

Jetzt werden wir einige wichtige Regualr Expression Funktionen anschauen die wir schon behandelt haben:

____
### grep()

Gebe den Index der Musterübereinstimmung zurück 

In [9]:
args(grep)

function (pattern, x, ignore.case = FALSE, perl = FALSE, value = FALSE, 
    fixed = FALSE, useBytes = FALSE, invert = FALSE) 
NULL

In [13]:
grep('A', c('A','B','C','D','A'))

[1] 1 5

### nchar()
Länge des Strings

In [14]:
args(nchar)

function (x, type = "chars", allowNA = FALSE, keepNA = FALSE) 
NULL

In [17]:
nchar('helloworld')

[1] 10

In [18]:
nchar('hello world')

[1] 11

### gsub()

Führe jetzt den Austausch des passenden Musters durch


In [19]:
args(gsub)

function (pattern, replacement, x, ignore.case = FALSE, perl = FALSE, 
    fixed = FALSE, useBytes = FALSE) 
NULL

In [20]:
gsub('pattern','replacement','hello have you seen the pattern here?')

[1] "hello have you seen the replacement here?"

## Text Manipulation

### paste()

mehrere Strings verbinden

In [32]:
print(paste('A','B','C',sep='...'))

[1] "A...B...C"


In [30]:
#help(paste)

### substr()

Gebe den Substring zurück der im gegebenen Abschnitt start:stop liegt

In [33]:
substr('abcdefg',start=2,stop = 5)

[1] "bcde"

### strsplit()

Teile den String in eine Liste von Substrings basierend auf einem anderen String x, der angibt wo der ursprüngliche String geteilt werden soll. 

In [49]:
strsplit('2016-01-23',split='-')

[[1]]
[1] "2016" "01"   "23"

_____
# NLP Wichtige Begriffe und Konzepte xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

Important Terms and Concepts

* Document - Das individuelle Textdokument 
* Corpus - Die Sammlung an Textdokumenten
* Bag-of-Words - Ungeordnete Sammlung an Wörtern
* n-grams - Zusammenhängende Sequenz von n Gegenständen von einer gegebenen Textsequenz 

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx contiguous sequence of n items from a given sequence of text (e.g. ['A','G','C',T']

* Stopwords - Stoppwörter sind sehr häufig auftauchenden Wörter einer Sprache, die gleichzeitig nicht wichtig für den wesentlichen Gehalt eines Texts sind. 
* Tokens - Kombination aus Zeichen (Wörter)
* Stemming - Prozess bei den wir Silben von Wörtern entfernen (z.b. run,runner,running auf das Basiswort "run" reduzieren)
* TF-IDF : "Term Frequency-Inver Document Frequency" ist ein statistisches Maß, dass dazu dient auszuwerten, wie wichtig ein Wort in einem Dokument in einem Corpus ist.
* Term Document Matrix - Representation einer Dokumentensammlung als Vektoren 

# Twitter Mining

Lass uns Twitter generellen Daten untersuchen:

## Schritt 1:  Libraries importieren

In [ ]:
library(twitteR)
library(tm)
library(wordcloud)
library(RColorBrewer)

## Schritt 2: Nach einem Thema auf Twitter suchen

Wir werden die twitteR Library verwenden um twitter zu untersuchen. Zu Beginn musst du dich verbinden, indem du die Autorizations Keys und die Tokens einstellst.

In [ ]:
setup_twitter_oauth(consumer_key, consumer_secret, access_token=NULL, access_secret=NULL)

Wir werden Twitter nach dem Wort 'soccer' untersuchen: 

In [ ]:
soccer.tweets <- searchTwitter("soccer", n=2000, lang="en")
soccer.text <- sapply(soccer.tweets, function(x) x$getText())

## Schritt 3: Daten Bereinigen

Wir entfernen die Emoticons und erstellen einen Corpus

In [ ]:
soccer.text <- iconv(soccer.text, 'UTF-8', 'ASCII') # Emoticons enfernen
soccer.corpus <- Corpus(VectorSource(soccer.text)) # Corpus erstellen

## Schritt 4: Erstelle die Document Term Matrix 

Wir werden ein paar Transformationen anwenden mithilfe der `TermDocumentMatrix` Funktion

In [ ]:
term.doc.matrix <- TermDocumentMatrix(soccer.corpus,
                                      control = list(removePunctuation = TRUE,
                                                     stopwords = c("soccer","http", stopwords("english")),
                                                     removeNumbers = TRUE,tolower = TRUE))

## Schritt 5: Überprüfung der Matrix

In [ ]:
head(term.doc.matrix)

In [ ]:
term.doc.matrix <- as.matrix(term.doc.matrix)

## Schritt 6: Anzahl der Wörter bekommen

In [ ]:
word.freqs <- sort(rowSums(term.doc.matrix), decreasing=TRUE) 
dm <- data.frame(word=names(word.freqs), freq=word.freqs)

## Schritt 7: Eine Word Cloud erstellen

In [ ]:
wordcloud(dm$word, dm$freq, random.order=FALSE, colors=brewer.pal(8, "Dark2"))

# Gut gemacht!